This is the setup cell

In [2]:
import datetime
import pathlib
import time
import math

import obspy
from obspy.signal import PPSD
import scipy
import matplotlib.pyplot as plt

#Run this line if you want interactive plots
%matplotlib qt

import numpy as np

import sprit

In [3]:
dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\TestDir'
#dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\Test12_AM.RAC84.00.2023-02-15_2132-2200.mseed'
trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'

params = sprit.input_param(  acq_date=46,#'2023-02-15',
                            #starttime = '21:32:00.00',
                            #endtime = '22:00:00.00',
                            tzone = 'UTC',#or 'local', #or 'UTC'
                            dst=False,
                            lon = -88.2290526,
                            lat =  40.1012122,
                            elevation = 755,
                            site = 'HVSR Site',
                            dataPath = dPath
                            )

No metadata file specified!
Using default metadata file for Raspberry Shake v.7 contained in repository at
 c:/Users/riley/LocalData/Github/SPRIT/resources/raspshake_metadata.inv


In [4]:
params = sprit.get_metadata(params)
stream = sprit.fetch_data(params, source='dir', trim_dir=trimDir)
params = sprit.generate_ppsds(params=params, stream=stream, ppsd_length=60, overlap=0.5, period_step_octaves=0.01)
hvsr_dict = sprit.process_hvsr(params=params, method=4, resample=False, smooth=10)
hvsr_dict = sprit.check_peaks(hvsr_dict=hvsr_dict)

Day of Year: 46


c:\ProgramData\Anaconda3\envs\seismic39\lib\site-packages\obspy\core\inventory\network.py:251: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


In [5]:
plt.ion()
sprit.hvplot(hvsr_dict, kind='HVSR', show=True)

In [7]:
from matplotlib import pyplot as plt
import matplotlib

class WindowBuilder:
    def __init__(, window):
        self.x0 = 0
        x1 = 1
        y0 = ax.get_ylim()[0]
        y1 = ax.get_ylim()[1]
        window = window
        self.paths = []
        self.vertices = [] 
        self.codes = []#window.get_paths()[0].codes
        
        #self.xs = list(window.get_xdata())
        #self.ys = list(window.get_ydata()) #may not need
        self.cid = window.figure.canvas.mpl_connect('button_press_event', self)
        self.clickNo = 0

    def __call__(self, event):
        if event.inaxes!=self.window.axes: return
        ax = event.inaxes
        self.y0 = ax.get_ylim()[0]
        self.y1 = ax.get_ylim()[1]
        if self.clickNo == 0:
            #y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 2)
            self.x0 = event.x
            self.clickNo += 1
            plt.axvline(self.x0, self.y0, self.y1, color='k')
            print('x0: ', event.xdata)
        else:
            self.x1 = event.x
            self.clickNo == 0
            plt.axvline(self.x0, self.y0, self.y1, color='k')
            print('x1: ', event.xdata)
            print(self.x0)
            print(self.x1)
            path_data = [
                (matplotlib.path.Path.MOVETO, (self.x0, self.y0)),
                (matplotlib.path.Path.LINETO, (self.x1, self.y0)),
                (matplotlib.path.Path.LINETO, (self.x1, self.y1)),
                (matplotlib.path.Path.LINETO, (self.x0, self.y1)),
                (matplotlib.path.Path.LINETO, (self.x0, self.y0)),
                (matplotlib.path.Path.CLOSEPOLY, (self.x0, self.y0)),
            ]
            codes, verts = zip(*path_data)
            path = matplotlib.path.Path(verts, codes)
            self.paths.append(path)
            for p in self.paths:
                print(p)
                patch = matplotlib.patches.PathPatch(p, facecolor='r', alpha=0.5)
                ax.add_patch(patch)
                plt.show()
            self.window.figure.canvas.draw()

fig, ax = plt.subplots()
ax.set_title('click to build line segments')
y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 1)
window = ax.fill_betweenx(y, [0], [0])
#print(window.get_paths()[0])
#line, = ax.plot([0], [0])  # empty line
linebuilder = WindowBuilder(window)

plt.show()

x0:  -0.03718429347461606
x1:  0.0039061766481121307
482
1039
Path(array([[ 4.820e+02, -5.500e-02],
       [ 1.039e+03, -5.500e-02],
       [ 1.039e+03,  5.500e-02],
       [ 4.820e+02,  5.500e-02],
       [ 4.820e+02, -5.500e-02],
       [ 4.820e+02, -5.500e-02]]), array([ 1,  2,  2,  2,  2, 79], dtype=uint8))


In [32]:
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

fig, ax = plt.subplots()
def plotfunc():
    t = np.arange(0.0, 1.0, 0.01)
    s = np.sin(2 * np.pi * t)
    ax.plot(t, s)
plotfunc()

clickNo = 0    
xWindows = []
pathList = []
windowDrawn = []
winArtist = []
lineArtist = []

def on_click(event, fig=fig, ax=ax):
    if event.button is MouseButton.RIGHT:
        remove_on_right(event)
    if event.button is MouseButton.LEFT:
        draw_boxes(event)

def draw_boxes(event, fig=fig, ax=ax):
    global clickNo
    global x0
    global xWindows
    global pathList
    global windowDrawn
    global winArtist
    global lineArtist

    if event.inaxes!=ax: return
    print(clickNo)
    y0, y1 = ax.get_ylim()
    
    if clickNo == 0:
        #y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 2)
        x0 = event.xdata
        clickNo += 1
        linArt = plt.axvline(x0, y0, y1, color='k')
        lineArtist.append([linArt, linArt])
    else:
        x1 = event.xdata
        clickNo = 0

        path_data = [
            (matplotlib.path.Path.MOVETO, (x0, y0)),
            (matplotlib.path.Path.LINETO, (x1, y0)),
            (matplotlib.path.Path.LINETO, (x1, y1)),
            (matplotlib.path.Path.LINETO, (x0, y1)),
            (matplotlib.path.Path.LINETO, (x0, y0)),
            (matplotlib.path.Path.CLOSEPOLY, (x0, y0)),
        ]

        codes, verts = zip(*path_data)
        path = matplotlib.path.Path(verts, codes)

        x_win = [x0, x1]
        xWindows.append(x_win)
        pathList.append(path)
        windowDrawn.append(False)
        winArtist.append(None)
        [lineArtist[-1].pop()]
        draw_windows(event=event, pathlist=pathList)
        linArt = plt.axvline(x1, y0, y1, color='k', zorder=100)
        lineArtist[-1].append(linArt)
    fig.canvas.draw() 

def draw_windows(event, pathlist, fig=fig, ax=ax):
    for i, p in enumerate(pathList):
        if windowDrawn[i]:
            pass
        else:
            patch = matplotlib.patches.PathPatch(p, facecolor='k', alpha=0.5)
            winArt = ax.add_patch(patch)
            windowDrawn[i] = True
            winArtist[i] = winArt
    if event.button is MouseButton.RIGHT:
        fig.canvas.draw() 

def remove_on_right(event, fig=fig, ax=ax, xWindows=xWindows):
    if xWindows is not None:
        for i, xWins in enumerate(xWindows):
            if event.xdata > xWins[0] and event.xdata < xWins[1]:
                xWindows.pop(i)
                pathList.pop(i)
                winArtist[i].remove()
                winArtist.pop(i)
                windowDrawn.pop(i)
                lineArtist[i][0].remove()
                lineArtist[i][1].remove()
                lineArtist.pop(i)
    fig.canvas.draw() 
    print(winArtist)
    #draw_windows(event, pathlist=pathList)

fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()

0
1
0
1
0
1
[<matplotlib.patches.PathPatch object at 0x00000209E0DC8D00>, <matplotlib.patches.PathPatch object at 0x00000209E1358820>]
[]


In [86]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
artists = []

# add some artists to the plot
for i in range(3):
    line, = ax.plot([0, 1], [i, i])
    artists.append(line)

# remove all artists from the plot
for artist in artists:
    artist.remove()

# update the plot
fig.canvas.draw()